# Pilot Ray Example

In [ ]:
# Pilot-Ray
import os, sys, socket
sys.path.insert(0, "../../..")
import pilot.streaming
import ray


#configure loggin
import logging
logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger("stevedore.extension").setLevel(logging.CRITICAL)
logging.getLogger("keystoneauth").setLevel(logging.CRITICAL)
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("asyncio").setLevel(logging.CRITICAL)

sys.modules['pilot.streaming']

RESOURCE_URL_HPC="slurm+ssh://login03.chn"
WORKING_DIRECTORY=os.path.join(os.environ["HOME"], "work")


## Setup Cluster

In [ ]:
# pilot_compute_description = {
#     "resource":RESOURCE_URL_HPC,
#     "working_directory": os.path.join(WORKING_DIRECTORY, "work"),
#     "number_of_nodes": 1, # used for SLURM allocation
#     "cores_per_node": 2, # used for Ray configuration
#     "gpus_per_node": 0,  # used for Ray configuration
#     "scheduler_script_commands": ["#SBATCH --constraint=cpu",
#                                   "#SBATCH --qos regular",
#                                   ],    
#     "walltime": 59,
#     "type":"ray",  
# }

pilot_compute_description = {
    "resource": RESOURCE_URL_HPC,
    "working_directory": WORKING_DIRECTORY,
    "number_cores": 1,
    "queue": "normal",
    "walltime": 5,
    "type": "ray",
    "project": "m4408",
    "os_ssh_keyfile": "~/.ssh/nersc",
    "scheduler_script_commands": ["#SBATCH --constraint=cpu"]
}

In [ ]:
ray_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description)
ray_pilot.wait()

## Ray Test

In [ ]:
ray_pilot.get_details()

In [ ]:
ray_client = ray_pilot.get_context()

In [ ]:

# create a remote function
@ray.remote
def f(x):
    return x * x

with ray_client:
    # call the remote function ten times in parallel
    # and return the results as a list
    print(ray.get([f.remote(i) for i in range(10)]))

# Shutdown Cluster

In [ ]:
ray_pilot.cancel()